Построить график курса доллара, евро или йены за желаемый период, начиная с 01.01.2019

In [62]:
import datetime
from dateutil.parser import parse 
import requests
import xml.etree.ElementTree as ET
import ipywidgets as ipw
import numpy as np
import matplotlib.pyplot as plt

In [263]:
def get_val(date_start, date_end, currency_key):
    
    date_start = date_start.isoformat()[0:11].split('-')
    date_start = f"{date_start[2]}/{date_start[1]}/{date_start[0]}"
    
    date_end = date_end.isoformat()[0:11].split('-')
    date_end = f"{date_end[2]}/{date_end[1]}/{date_end[0]}"

    currency = {'USD': 'R01235',
               'EUR': 'R01239',
               'JPY': 'R01820' 
               }
    params = {'date_req1': date_start,
             'date_req2' : date_end,
             'VAL_NM_RQ' : currency[currency_key]}
    url = 'http://www.cbr.ru/scripts/XML_dynamic.asp'
    r = requests.get(url, params=params)
    
    root = ET.fromstring(r.text)
    n = len(root)
    dates = [] 
    val_curs = np.empty((n, 1),  np.float32)
    
    for i, child in enumerate(root):
        date = child.attrib['Date']
        dates.append(parse(date, dayfirst=True).date())
        val_curs[i] = float(child.find('Value').text.replace(',','.'))
    
    return dates, val_curs;   

In [274]:
def plot_val_currency(date_start, date_end, currency):
    date, val_cur = get_val(date_start, date_end, currency[0])
    n = len(val_cur)
    val_currencyes = np.empty([n, len(currency)])
    val_currencyes[:, 0] = val_cur[:, 0]
    
    plt.plot(date, val_currencyes[:, 0])
    
    i = 1
    while i < len(currency): 
        _, val_cur = get_val(date_start, date_end, currency[i])
        val_currencyes[:, i] = val_cur[:, 0]
        plt.plot(date, val_currencyes[:, i])
        i += 1
    
    plt.grid()
    plt.xlim(min(date), max(date))
    plt.xticks(rotation=60)
    plt.xticks(minor=True)
    plt.ylabel('Курс в рублях')
    plt.xlabel('Дата')
    plt.legend(currency)
    
    for i, curr in enumerate(currency):
        val_max = round(np.max(val_currencyes[:, i]), 4)
        date_max = date[np.argmax(val_currencyes[:, i])]
        print(f'Максимальная цена {curr} = {val_max} была {date_max} ')
        
        val_min = round(np.min(val_currencyes[:, i]), 4)
        date_min = date[np.argmin(val_currencyes[:, i])]
        print(f'Минимальная цена {curr} = {val_min} была {date_min} \n ')

In [261]:
widget_currency = widgets.SelectMultiple(
    options=['USD', 'EUR', 'JPY'],
    value=['USD'],
    #rows=10,
    description='currency',
    disabled=False
)
widget_currency

SelectMultiple(description='currency', index=(0,), options=('USD', 'EUR', 'JPY'), value=('USD',))

In [31]:
widget_date_start = ipw.DatePicker(description = 'Pick date start:', value=datetime.date(2019, 1, 1))
widget_date_start

datetime.date(2019, 1, 1)

In [32]:
widget_date_end = ipw.DatePicker(description='Pick date end:', value=datetime.date.today())
widget_date_end

datetime.date(2023, 3, 9)

In [275]:
ip = ipw.interact(plot_val_currency, 
            date_start = widget_date_start,
            date_end = widget_date_end,
            currency = widget_currency)

interactive(children=(DatePicker(value=datetime.date(2019, 1, 1), description='Pick date start:', step=1), Dat…